<img src="https://github.com/Microsoft/sqlworkshops/blob/master/graphics/solutions-microsoft-logo-small.png?raw=true" alt="Microsoft">
<br>

# SQL Server 2019 big data cluster Tutorial
## 05 - Machine Learning with Python

In this tutorials you'll set up Machine Learning in SQL Server 2019 and create a Bulk-Scoring Routine.

More on this at [the Microsoft documentation site for Machine Learning Services.](https://docs.microsoft.com/en-us/sql/advanced-analytics/tutorials/sqldev-in-database-python-for-sql-developers?view=sql-server-2017)

In [0]:
/*
Run these commands where you have the kubectl utility installed. In this example, a temp directory
in the D: drive is used.

d:
cd \temp
curl -G "https://cs7a9736a9346a1x44c6xb00.blob.core.windows.net/backups/NYC.bak" -o NYC.bak
kubectl cp NYC.bak mssql-master-pool-0:/var/opt/mssql/data -c mssql-server -n <ReplaceWithClusterName>

*/

In [0]:
/* Add the Customer Databases for demonstrating Machine Learning */
USE [master]
RESTORE DATABASE [NYC] 
FROM  DISK = N'/var/opt/mssql/data/NYC.bak' 
WITH  FILE = 1
,  REPLACE
,  NOUNLOAD,  STATS = 5;
GO

In [1]:
/* Enable Machine Learning Services */
USE master;
GO

EXEC sp_configure 'external scripts enabled', 1;
GO
RECONFIGURE WITH OVERRIDE;
GO

Commands completed successfully.

Total execution time: 00:00:00.0203617

Configuration option 'external scripts enabled' changed from 0 to 1. Run the RECONFIGURE statement to install.

Total execution time: 00:00:00.1969723

Commands completed successfully.

Total execution time: 00:00:00.8810010

In [0]:
/* Load Prediction Data */
USE master;
GO

RESTORE DATABASE [NYC]
FROM  DISK = N'/var/opt/mssql/data/NYC.bak' 
WITH  FILE = 1
,  REPLACE
,  MOVE N'NYCTaxi_Sample' TO N'/var/opt/mssql/data/NYC.mdf'
,  MOVE N'NYCTaxi_Sample_log' TO N'/var/opt/mssql/data/NYC.ldf'
,  NOUNLOAD,  STATS = 5;
GO

In [2]:
/* Run some Python. This just returns a  a couple of library versions */
EXECUTE sp_execute_external_script
@language = N'Python',
@script = N'
import numpy as np
import pandas as pd
print(np.__version__)
print(pd.__version__)
',
@input_data_1 = N'SELECT 1 as Col1';
GO

STDOUT message(s) from external script: 
1.12.1
0.19.2

Total execution time: 00:00:17.8513455

In [16]:
USE NYC;
GO

/* Creates Plot Binary data */
DROP PROCEDURE IF EXISTS PyPlotMatplotlib;
GO

CREATE PROCEDURE [dbo].[PyPlotMatplotlib]
AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @query nvarchar(max) =
    N'SELECT cast(tipped as int) as tipped, tip_amount, fare_amount FROM [dbo].[nyctaxi_sample]'
    EXECUTE sp_execute_external_script
    @language = N'Python',
    @script = N'
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import pandas as pd
import pickle

fig_handle = plt.figure()
plt.hist(InputDataSet.tipped)
plt.xlabel("Tipped")
plt.ylabel("Counts")
plt.title("Histogram, Tipped")
plot0 = pd.DataFrame(data =[pickle.dumps(fig_handle)], columns =["plot"])
plt.clf()

plt.hist(InputDataSet.tip_amount)
plt.xlabel("Tip amount ($)")
plt.ylabel("Counts")
plt.title("Histogram, Tip amount")
plot1 = pd.DataFrame(data =[pickle.dumps(fig_handle)], columns =["plot"])
plt.clf()

plt.hist(InputDataSet.fare_amount)
plt.xlabel("Fare amount ($)")
plt.ylabel("Counts")
plt.title("Histogram, Fare amount")
plot2 = pd.DataFrame(data =[pickle.dumps(fig_handle)], columns =["plot"])
plt.clf()

plt.scatter( InputDataSet.fare_amount, InputDataSet.tip_amount)
plt.xlabel("Fare Amount ($)")
plt.ylabel("Tip Amount ($)")
plt.title("Tip amount by Fare amount")
plot3 = pd.DataFrame(data =[pickle.dumps(fig_handle)], columns =["plot"])
plt.clf()

OutputDataSet = plot0.append(plot1, ignore_index=True).append(plot2, ignore_index=True).append(plot3, ignore_index=True)
',
@input_data_1 = @query
WITH RESULT SETS ((plot varbinary(max)))
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0736030

Commands completed successfully.

Total execution time: 00:00:00.0934020

Commands completed successfully.

Total execution time: 00:00:00.0902700

In [17]:
/* Call that procedure */
EXEC [dbo].[PyPlotMatplotlib];
GO

Commands completed successfully.

Total execution time: 00:01:02.2179120

plot
...
...
...
...


In [5]:
USE NYC;
GO

/* Split the Sample */
DROP PROCEDURE IF EXISTS PyTrainTestSplit;
GO

CREATE PROCEDURE [dbo].[PyTrainTestSplit] (@pct int)
AS

DROP TABLE IF EXISTS dbo.nyctaxi_sample_training
SELECT * into nyctaxi_sample_training 
FROM nyctaxi_sample 
WHERE (ABS(CAST(BINARY_CHECKSUM(medallion,hack_license)  as int)) % 100) < @pct

DROP TABLE IF EXISTS dbo.nyctaxi_sample_testing
SELECT * into nyctaxi_sample_testing 
FROM nyctaxi_sample
WHERE (ABS(CAST(BINARY_CHECKSUM(medallion,hack_license)  as int)) % 100) > @pct
GO

Commands completed successfully.

Total execution time: 00:00:00.0737210

Commands completed successfully.

Total execution time: 00:00:00.0735570

Commands completed successfully.

Total execution time: 00:00:00.0902990

In [18]:
USE NYC;
GO 

/* Call proc using 60% Split */
EXEC PyTrainTestSplit 60
GO

Commands completed successfully.

Total execution time: 00:00:00.0775150

(1025684 rows affected)

(660029 rows affected)

Total execution time: 00:00:21.9306430

In [19]:
USE NYC;
GO

/* Create Linear Regression Model using SciKitLearn*/
DROP PROCEDURE IF EXISTS PyTrainScikit;
GO

CREATE PROCEDURE [dbo].[PyTrainScikit] (@trained_model varbinary(max) OUTPUT)
AS
BEGIN
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import numpy
import pickle
from sklearn.linear_model import LogisticRegression

##Create SciKit-Learn logistic regression model
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

SKLalgo = LogisticRegression()
logitObj = SKLalgo.fit(X, y)

##Serialize model
trained_model = pickle.dumps(logitObj)
',
@input_data_1 = N'
select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance, 
dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
from nyctaxi_sample_training
',
@input_data_1_name = N'InputDataSet',
@params = N'@trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT;
;
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.0745870

Commands completed successfully.

Total execution time: 00:00:00.0874570

Commands completed successfully.

Total execution time: 00:00:00.0890390

In [26]:
USE NYC;
GO 

/* 
Versioning:

ALTER TABLE dbo.nyc_taxi_models
ADD ModelVersion DATETIME;
GO
*/

SELECT * FROM  [NYC].[dbo].[nyc_taxi_models]
TRUNCATE TABLE [NYC].[dbo].[nyc_taxi_models]

/* Store the models. You could version, tag, or stamp here if you want */
DECLARE @model VARBINARY(MAX)
EXEC PyTrainScikit @model OUTPUT
INSERT INTO nyc_taxi_models (name, model, ModelVersion) VALUES('SciKit_model', @model, GETDATE());
GO

Commands completed successfully.

Total execution time: 00:00:00.0744610

(3 rows affected)

(1 row affected)

Total execution time: 00:00:22.5621200

model name ModelVersion 0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E6C6F6769737469630A4C6F67697374696352656772657373696F6E0A7100298171017D71022858100000005F736B6C6561726E5F76657273696F6E71035806000000302E31382E3171045806000000736F6C766572710558090000006C69626C696E6561727106580B0000006D756C74695F636C617373710758030000006F76727108580C00000072616E646F6D5F737461746571094E58070000006E5F697465725F710A636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A710B636E756D70790A6E6461727261790A710C4B0085710D430162710E87710F527110284B014B01857111636E756D70790A64747970650A71125802000000693471134B004B01877114527115284B0358010000003C71164E4E4E4AFFFFFFFF4AFFFFFFFF4B007471176289430417000000711874711962580C000000636C6173735F776569676874711A4E5808000000636C61737365735F711B680B680C4B0085711C680E87711D52711E284B014B0285711F68158943080000000001000000712074712162580A000000696E746572636570745F7122680B680C4B00857123680E877124527125284B014B0185712668125802000000663871274B004B01877128527129284B0368164E4E4E4AFFFFFFFF4AFFFFFFFF4B0074712A62894308831B1D5A762EA13F712B74712C62580100000043712D473FF0000000000000580D0000006669745F696E74657263657074712E885803000000746F6C712F473F1A36E2EB1C432D58080000006D61785F6974657271304B64580A0000007761726D5F737461727471318958060000006E5F6A6F627371324B01580700000070656E616C7479713358020000006C32713458040000006475616C7135895805000000636F65665F7136680B680C4B00857137680E877138527139284B014B014B0486713A6829894320F2E8A75D3203A1BF53E4B0B17AA8A53F2F7F8DC9FED582BEB8A81FEDEB68123F713B74713C625807000000766572626F7365713D4B005811000000696E746572636570745F7363616C696E67713E4B0175622E SciKit_model 2019-04-09 18:45:51.413 0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E6C6F6769737469630A4C6F67697374696352656772657373696F6E0A7100298171017D71022858040000006475616C710389580700000070656E616C7479710458020000006C3271055803000000746F6C7106473F1A36E2EB1C432D5801000000437107473FF0000000000000580D0000006669745F696E74657263657074710888580C00000072616E646F6D5F737461746571094E58060000006E5F6A6F6273710A4B01580A000000696E746572636570745F710B636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A710C636E756D70790A6E6461727261790A710D4B0085710E430162710F877110527111284B014B01857112636E756D70790A64747970650A71135802000000663871144B004B01877115527116284B0358010000003C71174E4E4E4AFFFFFFFF4AFFFFFFFF4B0074711862894308831B1D5A762EA13F711974711A62580C000000636C6173735F776569676874711B4E5807000000766572626F7365711C4B0058100000005F736B6C6561726E5F76657273696F6E711D5806000000302E31382E31711E5806000000736F6C766572711F58090000006C69626C696E6561727120580A0000007761726D5F7374617274712189580B0000006D756C74695F636C617373712258030000006F767271235805000000636F65665F7124680C680D4B00857125680F877126527127284B014B014B048671286816894320F2E8A75D3203A1BF53E4B0B17AA8A53F2F7F8DC9FED582BEB8A81FEDEB68123F712974712A625811000000696E746572636570745F7363616C696E67712B4B015808000000636C61737365735F712C680C680D4B0085712D680F87712E52712F284B014B0285713068135802000000693471314B004B01877132527133284B0368174E4E4E4AFFFFFFFF4AFFFFFFFF4B0074713462894308000000000100000071357471366258080000006D61785F6974657271374B6458070000006E5F697465725F7138680C680D4B00857139680F87713A52713B284B014B0185713C683389430417000000713D74713E6275622E SciKit_model 2019-04-09 18:46:25.957 0x8003637265766F7363616C6570792E66756E6374696F6E732E52784C6F6769740A52784C6F676974526573756C74730A7100298171017D71022858100000005F636F6D707574655F636F6E746578747103637265766F7363616C6570792E636F6D70757465636F6E746578742E52784C6F63616C5365710A52784C6F63616C5365710A7104298171057D710628580C0000005F6465736372697074696F6E710758080000004C6F63616C5365717108580D0000005F6175746F5F636C65616E757071098858090000005F72656D6F74654964710A4AFFFFFFFF58030000005F6964710B58200000006430386562656134356166373131653961643038386532623163363166303164710C580F0000005F636F6E736F6C655F6F7574707574710D8958080000005F76657273696F6E710E5805000000312E302D31710F756258080000005F666F726D756C617110584E000000746970706564207E2070617373656E

In [10]:
/* Train using RevoScalePy */
USE NYC;
GO

DROP PROCEDURE IF EXISTS TrainTipPredictionModelRxPy;
GO

CREATE PROCEDURE [dbo].[TrainTipPredictionModelRxPy] (@trained_model varbinary(max) OUTPUT)
AS
BEGIN
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import numpy
import pickle
from revoscalepy.functions.RxLogit import rx_logit

## Create a logistic regression model using rx_logit function from revoscalepy package
logitObj = rx_logit("tipped ~ passenger_count + trip_distance + trip_time_in_secs + direct_distance", data = InputDataSet);

## Serialize model
trained_model = pickle.dumps(logitObj)
',
@input_data_1 = N'
select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance, 
dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
from nyctaxi_sample_training
',
@input_data_1_name = N'InputDataSet',
@params = N'@trained_model varbinary(max) OUTPUT',
@trained_model = @trained_model OUTPUT;
;
END;
GO

Commands completed successfully.

Total execution time: 00:00:00.0741140

Commands completed successfully.

Total execution time: 00:00:00.0736050

Commands completed successfully.

Total execution time: 00:00:00.1079610

In [27]:
USE NYC;
GO

/* Now save that model out */
DECLARE @model VARBINARY(MAX);
EXEC TrainTipPredictionModelRxPy @model OUTPUT;
INSERT INTO nyc_taxi_models (name, model, ModelVersion) VALUES('revoscalepy_model', @model, GETDATE());
GO

SELECT * FROM  [NYC].[dbo].[nyc_taxi_models]


Commands completed successfully.

Total execution time: 00:00:00.0761130

STDOUT message(s) from external script: 
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.027 seconds 

Starting values (iteration 1) time: 0.035 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.064 seconds 

Iteration 2 time: 0.072 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.063 seconds 

Iteration 3 time: 0.069 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.062 seconds 

Iteration 4 time: 0.069 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.060 seconds 

Iteration 5 time: 0.066 secs.
Rows Read: 1025684, Total Rows Processed: 1025684
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation.
, Total Chunk Time: 0.068 seconds 

Iteration 6 time: 0.075 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.063 seconds 

Iteration 7 time: 0.070 secs.
Rows Read: 1025684, Total Rows Processed: 1025684

STDOUT message(s) from external script: 
 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation. (similar message repeated 3 times)
, Total Chunk Time: 0.064 seconds 

Iteration 8 time: 0.071 secs.
Rows Read: 1025684, Total Rows Processed: 1025684, Total Chunk Time: 0.069 seconds 

Iteration 9 time: 0.075 secs.

Elapsed computation time: 0.604 secs.


 Warning: One or more fitted probabilities were numerically 0 or 1 during estimation. (similar message repeated 4 times)

(1 row affected)

Total execution time: 00:00:17.3353210

(2 rows affected)

Total execution time: 00:00:00.0761580

model name ModelVersion 0x800363736B6C6561726E2E6C696E6561725F6D6F64656C2E6C6F6769737469630A4C6F67697374696352656772657373696F6E0A7100298171017D71022858040000006475616C71038958080000006D61785F6974657271044B64580A000000696E746572636570745F7105636E756D70792E636F72652E6D756C746961727261790A5F7265636F6E7374727563740A7106636E756D70790A6E6461727261790A71074B00857108430162710987710A52710B284B014B0185710C636E756D70790A64747970650A710D58020000006638710E4B004B0187710F527110284B0358010000003C71114E4E4E4AFFFFFFFF4AFFFFFFFF4B0074711262894308831B1D5A762EA13F711374711462580A0000007761726D5F7374617274711589580C00000072616E646F6D5F737461746571164E580C000000636C6173735F77656967687471174E5807000000766572626F736571184B0058070000006E5F697465725F7119680668074B0085711A680987711B52711C284B014B0185711D680D58020000006934711E4B004B0187711F527120284B0368114E4E4E4AFFFFFFFF4AFFFFFFFF4B007471216289430417000000712274712362580D0000006669745F696E746572636570747124885803000000746F6C7125473F1A36E2EB1C432D5808000000636C61737365735F7126680668074B008571276809877128527129284B014B0285712A68208943080000000001000000712B74712C625811000000696E746572636570745F7363616C696E67712D4B015806000000736F6C766572712E58090000006C69626C696E656172712F5805000000636F65665F7130680668074B008571316809877132527133284B014B014B048671346810894320F2E8A75D3203A1BF53E4B0B17AA8A53F2F7F8DC9FED582BEB8A81FEDEB68123F713574713662580B0000006D756C74695F636C617373713758030000006F76727138580700000070656E616C7479713958020000006C32713A58060000006E5F6A6F6273713B4B01580100000043713C473FF000000000000058100000005F736B6C6561726E5F76657273696F6E713D5806000000302E31382E31713E75622E SciKit_model 2019-04-09 18:49:24.260 0x8003637265766F7363616C6570792E66756E6374696F6E732E52784C6F6769740A52784C6F676974526573756C74730A7100298171017D71022858070000005F706172616D7371037D710428580B0000007573657250726F66696C657105586B0000002F7661722F6F70742F6D7373716C2D657874656E736962696C6974792F646174612F39636332343438352D656662312D343364302D393534392D6232313730353265376463632F34423643314141442D393041422D343344352D383339372D3931424635374646413442307106580E00000053686F77497465726174696F6E73710789581600000053686F7744657461696C6564497465726174696F6E73710889580D0000007472616E73666F726D5661727371094E581A0000004F626A65637469766546756E6374696F6E546F6C6572616E6365710A473E45798EE2308C3A580F0000004375626550726564696374696F6E73710B895803000000746D70710C586B0000002F7661722F6F70742F6D7373716C2D657874656E736962696C6974792F646174612F39636332343438352D656662312D343364302D393534392D6232313730353265376463632F34423643314141442D393041422D343344352D383339372D393142463537464641344230710D5807000000566572626F7365710E4B00580800000072436F6E64546F6C710F473D06849B86A12B9B5807000000466F726D756C617110584E000000746970706564207E2070617373656E6765725F636F756E74202B20747269705F64697374616E6365202B20747269705F74696D655F696E5F73656373202B206469726563745F64697374616E63657111580800000064726F704D61696E7112885807000000636F7644617461711389580E000000696E697469616C5F76616C75657371144E580E000000436F656666546F6C6572616E63657115473EB0C6F7A0B5ED8D58100000007472616E73666F726D4F626A6563747371164E580D0000004D6178497465726174696F6E7371174B19580D0000002E2E7278446174614672616D6571184E580C000000726F7753656C656374696F6E71194E580D0000007472616E73666F726D46756E63711A4E5807000000636F76436F6566711B8958110000004772616469656E74546F6C6572616E6365711C473EB0C6F7A0B5ED8D580900000064726F704669727374711D89580E0000005265706F727450726F6772657373711E88580E00000072656D6F76654D697373696E6773711F885808000000726561645661727371205D71212858060000007469707065647122580B000000666172655F616D6F756E747123580F00000070617373656E6765725F636F756E7471245811000000747269705F74696D655F696E5F736563737125580D000000747269705F64697374616E63657126580F0000006469726563745F64697374616E6365712765581500000062557365725365744D6178497465726174696F6E737128895806000000766172546F6C7129473D719799812DEA115806000000466164644350712A63636F6C6C656374696F6E730A4F726465726564446963740A712B2952712C580F00000062436F6D707574654C6F7748696768712D8873580500

In [12]:
USE NYC;
GO 

/* Create Scoring function from SciKit Learn */
DROP PROCEDURE IF EXISTS PredictTipSciKitPy;
GO

CREATE PROCEDURE [dbo].[PredictTipSciKitPy] (@model varchar(50), @inquery nvarchar(max))
AS
BEGIN
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
from sklearn import metrics

mod = pickle.loads(lmodel2)
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

probArray = mod.predict_proba(X)
probList = []
for i in range(len(probArray)):
  probList.append((probArray[i])[1])

probArray = numpy.asarray(probList)
fpr, tpr, thresholds = metrics.roc_curve(y, probArray)
aucResult = metrics.auc(fpr, tpr)
print ("AUC on testing data is: " + str(aucResult))

OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',	
  @input_data_1 = @inquery,
  @input_data_1_name = N'InputDataSet',
  @params = N'@lmodel2 varbinary(max)',
  @lmodel2 = @lmodel2
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0734440

Commands completed successfully.

Total execution time: 00:00:00.0746960

Commands completed successfully.

Total execution time: 00:00:00.0900500

In [13]:
USE NYC;
GO

/* Score with RevoScalePy */
DROP PROCEDURE IF EXISTS PredictTipRxPy;
GO

CREATE PROCEDURE [dbo].[PredictTipRxPy] (@model varchar(50), @inquery nvarchar(max))
AS
BEGIN
DECLARE @lmodel2 varbinary(max) = (select model from nyc_taxi_models where name = @model);
EXEC sp_execute_external_script 
  @language = N'Python',
  @script = N'
import pickle;
import numpy;
from sklearn import metrics
from revoscalepy.functions.RxPredict import rx_predict;

mod = pickle.loads(lmodel2)
X = InputDataSet[["passenger_count", "trip_distance", "trip_time_in_secs", "direct_distance"]]
y = numpy.ravel(InputDataSet[["tipped"]])

probArray = rx_predict(mod, X)
probList = probArray["tipped_Pred"].values 

probArray = numpy.asarray(probList)
fpr, tpr, thresholds = metrics.roc_curve(y, probArray)
aucResult = metrics.auc(fpr, tpr)
print ("AUC on testing data is: " + str(aucResult))

OutputDataSet = pandas.DataFrame(data = probList, columns = ["predictions"])
',	
  @input_data_1 = @inquery,
  @input_data_1_name = N'InputDataSet',
  @params = N'@lmodel2 varbinary(max)',
  @lmodel2 = @lmodel2
WITH RESULT SETS ((Score float));
END
GO

Commands completed successfully.

Total execution time: 00:00:00.0765730

Commands completed successfully.

Total execution time: 00:00:00.0769170

Commands completed successfully.

Total execution time: 00:00:00.1109080

In [14]:
USE NYC;
GO

/* Run a bulk score using SciKitLearn and the proc for that */
DECLARE @query_string nvarchar(max) -- Specify input query
  SET @query_string='
  select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance,
  dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
  from nyctaxi_sample_testing'
EXEC [dbo].[PredictTipSciKitPy] 'SciKit_model', @query_string; 

Commands completed successfully.

Total execution time: 00:00:00.0735390

: Msg 39004, Level 16, State 20, Line 3
A 'Python' script error occurred during execution of 'sp_execute_external_script' with HRESULT 0x80004004.

: Msg 39019, Level 16, State 2, Line 3
An external script error occurred: 

Error in execution.  Check the output for more information.
Traceback (most recent call last):
  File "<string>", line 5, in <module>
  File "/var/opt/mssql-extensibility/data/10f31ef8-195b-4998-96d7-f484eafeaf1e/741039BE-686E-4DE7-B072-61E7DFFBB4FC/sqlindb_0.py", line 38, in transform
    mod = pickle.loads(lmodel2)
TypeError: a bytes-like object is required, not 'NoneType'

SqlSatelliteCall error: Error in execution.  Check the output for more information.

STDOUT message(s) from external script: 
SqlSatelliteCall function failed. Please see the console output for more information.
Traceback (most recent call last):
 File "/opt/mssql/mlservices/libraries/PythonServer/revoscalepy/computecontext/RxInSqlServer.py", line 605, in rx_sql_satellite_call
 rx_native_call("SqlSatelliteCall", params)
 File "/opt/mssql/mlservices/libraries/PythonServer/revoscalepy/RxSerializable.py", line 375, in rx_native_call
 ret = px_call(functionname, params)
RuntimeError: revoscalepy function failed.

Total execution time: 00:00:11.6060510

Score


In [15]:
USE NYC;
GO

/* Do the same from RevoScalePy */
DECLARE @query_string nvarchar(max) -- Specify input query
  SET @query_string='
  select tipped, fare_amount, passenger_count, trip_time_in_secs, trip_distance,
  dbo.fnCalculateDistance(pickup_latitude, pickup_longitude,  dropoff_latitude, dropoff_longitude) as direct_distance
  from nyctaxi_sample_testing'
EXEC [dbo].[PredictTipRxPy] 'revoscalepy_model', @query_string; 

Commands completed successfully.

Total execution time: 00:00:00.0743710

STDOUT message(s) from external script: 
Rows Read: 660029, Total Rows Processed: 660029, Total Chunk Time: 0.021 seconds 
AUC on testing data is: 0.56502019161

(660029 rows affected)

Total execution time: 00:00:15.5037590

Displaying Top 5000 rows.

Score
0.508539321169836
0.508544730279092
0.508543254708527
0.508547740823509
0.508541109968948
0.508540714804605
0.50854041676457
0.508542904457477
0.508539051233405
0.508541680472037
